# Scratchpad for testing out graphlab with project data

In [1]:
import json
import numpy as np
import graphlab
import pandas as pd
import matplotlib.pyplot as plt

[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1467160691.log


This non-commercial license of GraphLab Create is assigned to windsurf_dean@yahoo.com and will expire on May 26, 2017. For commercial licensing options, visit https://dato.com/buy/.


### Load in user-item-rating data...

In [2]:
# create Pandas DF with data pulled from Postgres
df = pd.read_csv('data_analysis/user_reviews.csv', header=None,
                 names=['user_id', 'business_id', 'bus_name', 'stars', 'locale'])

In [3]:
df.head()

,user_id,business_id,bus_name,stars,locale
0,Ouus3lC3Xk3YRGx4ptd_4A,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,2,"Las Vegas, NV"
1,dz_sBsL69aWzsxksT2O_Mg,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,2,"Las Vegas, NV"
2,zyI-u0C1YOzp5v1j2wuCOQ,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,3,"Las Vegas, NV"
3,vyfsQo-estP8EfiIFMsL6g,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,3,"Las Vegas, NV"
4,lxZSVeJz6KEBW1nlA3JKJg,DW6SI2KxcfXBHeo9jEr4lQ,Pyramid Cafe,1,"Las Vegas, NV"


In [4]:
# store data in Graphlab's SFrame type
sf = graphlab.SFrame(df[['user_id', 'business_id', 'stars']])

In [5]:
sf.head()

user_id,business_id,stars
Ouus3lC3Xk3YRGx4ptd_4A,DW6SI2KxcfXBHeo9jEr4lQ,2
dz_sBsL69aWzsxksT2O_Mg,DW6SI2KxcfXBHeo9jEr4lQ,2
zyI-u0C1YOzp5v1j2wuCOQ,DW6SI2KxcfXBHeo9jEr4lQ,3
vyfsQo-estP8EfiIFMsL6g,DW6SI2KxcfXBHeo9jEr4lQ,3
lxZSVeJz6KEBW1nlA3JKJg,DW6SI2KxcfXBHeo9jEr4lQ,1
CbMCOeeHVeafcD-6-CJrPg,DW6SI2KxcfXBHeo9jEr4lQ,4
Z8DyUMojRhFbLyHBymM_rA,DW6SI2KxcfXBHeo9jEr4lQ,2
sXWvDlMQspjJ1zzr3cEM7Q,DW6SI2KxcfXBHeo9jEr4lQ,2
G8RaaiVzbyRvsBmoUx0VeA,DW6SI2KxcfXBHeo9jEr4lQ,1
f0isFKnJVZVmpdCPNd8Gfw,DW6SI2KxcfXBHeo9jEr4lQ,1


In [6]:
sf.column_names

<bound method SFrame.column_names of Columns:
	user_id	str
	business_id	str
	stars	int

Rows: 1363242

Data:
+------------------------+------------------------+-------+
|        user_id         |      business_id       | stars |
+------------------------+------------------------+-------+
| Ouus3lC3Xk3YRGx4ptd_4A | DW6SI2KxcfXBHeo9jEr4lQ |   2   |
| dz_sBsL69aWzsxksT2O_Mg | DW6SI2KxcfXBHeo9jEr4lQ |   2   |
| zyI-u0C1YOzp5v1j2wuCOQ | DW6SI2KxcfXBHeo9jEr4lQ |   3   |
| vyfsQo-estP8EfiIFMsL6g | DW6SI2KxcfXBHeo9jEr4lQ |   3   |
| lxZSVeJz6KEBW1nlA3JKJg | DW6SI2KxcfXBHeo9jEr4lQ |   1   |
| CbMCOeeHVeafcD-6-CJrPg | DW6SI2KxcfXBHeo9jEr4lQ |   4   |
| Z8DyUMojRhFbLyHBymM_rA | DW6SI2KxcfXBHeo9jEr4lQ |   2   |
| sXWvDlMQspjJ1zzr3cEM7Q | DW6SI2KxcfXBHeo9jEr4lQ |   2   |
| G8RaaiVzbyRvsBmoUx0VeA | DW6SI2KxcfXBHeo9jEr4lQ |   1   |
| f0isFKnJVZVmpdCPNd8Gfw | DW6SI2KxcfXBHeo9jEr4lQ |   1   |
+------------------------+------------------------+-------+
[1363242 rows x 3 columns]
Note: Only the head of t

### Load in item_data, for side_data_factorization...

In [8]:
pd.__version__

u'0.18.0'

In [19]:
# create Pandas DF from json business data
fname = 'data_analysis/test_data/test100_business.json'
with open(fname) as js_file:
#     for js_line in js_file:
    js_line = js_file.readline()
    js = json.loads(js_line)
        


# pd.io.json.json_normalize()
# dfb = pd.read_json('data_analysis/test_data/test100_business.json', orient='values')

# df = pd.read_csv('data_analysis/user_reviews.csv', header=None,
#                  names=['user_id', 'business_id', 'bus_name', 'stars', 'locale'])

In [24]:
print js['business_id']
print js['categories']
print js['attributes']

5UmKMjUEUNdYWqANhGckJw
[u'Fast Food', u'Restaurants']
{u'Take-out': True, u'Drive-Thru': False, u'Outdoor Seating': False, u'Caters': False, u'Noise Level': u'average', u'Parking': {u'garage': False, u'street': False, u'validated': False, u'lot': False, u'valet': False}, u'Delivery': False, u'Attire': u'casual', u'Has TV': False, u'Price Range': 1, u'Good For': {u'dessert': False, u'latenight': False, u'lunch': False, u'dinner': False, u'breakfast': False, u'brunch': False}, u'Takes Reservations': False, u'Ambience': {u'romantic': False, u'intimate': False, u'classy': False, u'hipster': False, u'divey': False, u'touristy': False, u'trendy': False, u'upscale': False, u'casual': False}, u'Waiter Service': False, u'Accepts Credit Cards': True, u'Good for Kids': True, u'Good For Groups': True, u'Alcohol': u'none'}


In [ ]:
# create the recommender (will train during this step)
rec = graphlab.recommender.factorization_recommender.create(
            sf,
            user_id='user_id',
            item_id='business_id',
            target='stars',
            solver='als',
            side_data_factorization=False)

In [ ]:
rec = graphlab.recommender.create(
            sf,
            user_id='user_id',
            item_id='business_id',
            target ='stars')

In [ ]:
# evaluate 1 datapoint (user=1, business=100) for potential rating
one_datapoint_sf = graphlab.SFrame({'user_id': [1], 'business_id': [100]})
print "stars:", rec.predict(one_datapoint_sf)[0]   # 4.879

In [ ]:
rec.list_fields()

In [ ]:
rec.get('coefficients')['intercept']

In [ ]:
# rec['coefficients']  # equivalent to: rec.get('coefficients')
rec.get('coefficients')['user_id']

In [ ]:
# rec['coefficients']  # equivalent to: rec.get('coefficients')
rec.get('coefficients')['business_id']

In [ ]:
business_sf = rec['coefficients']['business_id']
print len(business_sf)                     # 1682
print len(business_sf['factors'][0])       # 8
user_sf = rec['coefficients']['user_id']
print len(user_sf)                      # 943
print len(user_sf['factors'][0])        # 8

In [ ]:
import os

In [ ]:
os.path.getsize()

In [ ]:
import progressbar

In [ ]:
progressbar.__version__

In [ ]:
bar = progressbar.ProgressBar